# Tuning Neural Networks with Normalization - Lab 

## Introduction

In this lab you'll build a neural network to perform a regression task.

It is worth noting that getting regression to work with neural networks can be comparatively difficult because the output is unbounded ($\hat y$ can technically range from $-\infty$ to $+\infty$), and the models are especially prone to exploding gradients. This issue makes a regression exercise the perfect learning case for tinkering with normalization and optimization strategies to ensure proper convergence!

## Objectives

In this lab you will: 

- Fit a neural network to normalized data 
- Implement and observe the impact of various initialization techniques 
- Implement and observe the impact of various optimization techniques 

## Load the data 

First, run the following cell to import all the neccessary libraries and classes you will need in this lab. 

In [1]:
# Necessary libraries and classes
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras import initializers
from keras import layers
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
from keras import optimizers
from sklearn.model_selection import train_test_split

import warnings
warnings.filterwarnings('ignore')

In this lab, you'll be working with the housing prices data you saw in an earlier section. However, we did a lot of preprocessing for you so you can focus on normalizing numeric features and building neural network models! The following preprocessing steps were taken (all the code can be found in the `data_preprocessing.ipynb` notebook in this repository): 

- The data was split into the training, validate, and test sets 
- All the missing values in numeric columns were replaced by the median of those columns 
- All the missing values in catetgorical columns were replaced with the word 'missing' 
- All the categorical columns were one-hot encoded 

Run the following cells to import the train, validate, and test sets:  

In [2]:
# Load all numeric features
X_train_numeric = pd.read_csv('data/X_train_numeric.csv')
X_val_numeric = pd.read_csv('data/X_val_numeric.csv')
X_test_numeric = pd.read_csv('data/X_test_numeric.csv')

# Load all categorical features
X_train_cat = pd.read_csv('data/X_train_cat.csv')
X_val_cat = pd.read_csv('data/X_val_cat.csv')
X_test_cat = pd.read_csv('data/X_test_cat.csv')

# Load all targets
y_train = pd.read_csv('data/y_train.csv')
y_val = pd.read_csv('data/y_val.csv')
y_test = pd.read_csv('data/y_test.csv')

In [3]:
# Combine all features
X_train = pd.concat([X_train_numeric, X_train_cat], axis=1)
X_val = pd.concat([X_val_numeric, X_val_cat], axis=1)
X_test = pd.concat([X_test_numeric, X_test_cat], axis=1)

# Number of features
n_features = X_train.shape[1]

As a refresher, preview the training data: 

In [4]:
# Preview the data
X_train.head()

,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,...,SaleType_ConLw,SaleType_New,SaleType_Oth,SaleType_WD,SaleCondition_Abnorml,SaleCondition_AdjLand,SaleCondition_Alloca,SaleCondition_Family,SaleCondition_Normal,SaleCondition_Partial
0,80.0,69.0,21453.0,6.0,5.0,1969.0,1969.0,0.0,938.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
1,60.0,79.0,12420.0,7.0,5.0,2001.0,2001.0,0.0,666.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
2,20.0,75.0,9742.0,8.0,5.0,2002.0,2002.0,281.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
3,120.0,39.0,5389.0,8.0,5.0,1995.0,1996.0,0.0,1180.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
4,60.0,85.0,11003.0,10.0,5.0,2008.0,2008.0,160.0,765.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0


## Build a Baseline Model

Building a naive baseline model to compare performance against is a helpful reference point. From there, you can then observe the impact of various tunning procedures which will iteratively improve your model. So, let's do just that! 

In the cell below: 

- Add an input layer with `n_features` units 
- Add two hidden layers, one with 100 and the other with 50 units (make sure you use the `'relu'` activation function) 
- Add an output layer with 1 unit and `'linear'` activation 
- Compile and fit the model 

In [5]:
np.random.seed(123)
baseline_model = Sequential()

# Hidden layer with 100 units
baseline_model.add(layers.Dense(100, activation='relu', input_shape=(n_features,)))

# Hidden layer with 50 units
baseline_model.add(layers.Dense(50, activation='relu'))

# Output layer
baseline_model.add(layers.Dense(1, activation='linear'))

# Compile the model
baseline_model.compile(optimizer='SGD', 
                       loss='mse', 
                       metrics=['mse'])

# Train the model
baseline_model.fit(X_train, 
                   y_train, 
                   batch_size=32, 
                   epochs=150, 
                   validation_data=(X_val, y_val))

Epoch 1/150
33/33 [==============================] - 0s 10ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 2/150
33/33 [==============================] - 0s 4ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 3/150
33/33 [==============================] - 0s 4ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 4/150
33/33 [==============================] - 0s 4ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 5/150
33/33 [==============================] - 0s 4ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 6/150
33/33 [==============================] - 0s 4ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 7/150
33/33 [==============================] - 0s 4ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 8/150
33/33 [==============================] - 0s 4ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 9/150
33/33 [============================

33/33 [==============================] - 0s 6ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 70/150
33/33 [==============================] - 0s 5ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 71/150
33/33 [==============================] - 0s 5ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 72/150
33/33 [==============================] - 0s 5ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 73/150
33/33 [==============================] - 0s 5ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 74/150
33/33 [==============================] - 0s 5ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 75/150
33/33 [==============================] - 0s 5ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 76/150
33/33 [==============================] - 0s 6ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 77/150
33/33 [==============================] -

33/33 [==============================] - 0s 5ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 138/150
33/33 [==============================] - 0s 5ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 139/150
33/33 [==============================] - 0s 5ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 140/150
33/33 [==============================] - 0s 5ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 141/150
33/33 [==============================] - 0s 5ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 142/150
33/33 [==============================] - 0s 5ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 143/150
33/33 [==============================] - 0s 5ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 144/150
33/33 [==============================] - 0s 5ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 145/150
33/33 [=========================

> _**Notice this extremely problematic behavior: all the values for training and validation loss are "nan". This indicates that the algorithm did not converge. The first solution to this is to normalize the input. From there, if convergence is not achieved, normalizing the output may also be required.**_ 

## Normalize the Input Data 

It's now time to normalize the input data. In the cell below: 

- Assign the column names of all numeric columns to `numeric_columns` 
- Instantiate a `StandardScaler` 
- Fit and transform `X_train_numeric`. Make sure you convert the result into a DataFrame (use `numeric_columns` as the column names) 
- Transform validate and test sets (`X_val_numeric` and `X_test_numeric`), and convert these results into DataFrames as well 
- Use the provided to combine the scaled numerical and categorical features 

In [6]:
# Numeric column names
numeric_columns = X_train_numeric.columns 

# Instantiate StandardScaler
ss_X = StandardScaler()

# Fit and transform train data
X_train_scaled = pd.DataFrame(ss_X.fit_transform(X_train_numeric), columns=numeric_columns)

# Transform validate and test data
X_val_scaled = pd.DataFrame(ss_X.transform(X_val_numeric), columns=numeric_columns)
X_test_scaled = pd.DataFrame(ss_X.transform(X_test_numeric), columns=numeric_columns)

# Combine the scaled numerical features and categorical features
X_train = pd.concat([X_train_scaled, X_train_cat], axis=1)
X_val = pd.concat([X_val_scaled, X_val_cat], axis=1)
X_test = pd.concat([X_test_scaled, X_test_cat], axis=1)

Now run the following cell to compile a neural network model (with the same architecture as before): 

In [7]:
# Model with all normalized inputs
np.random.seed(123)
normalized_input_model = Sequential()
normalized_input_model.add(layers.Dense(100, activation='relu', input_shape=(n_features,)))
normalized_input_model.add(layers.Dense(50, activation='relu'))
normalized_input_model.add(layers.Dense(1, activation='linear'))

# Compile the model
normalized_input_model.compile(optimizer='SGD', 
                               loss='mse', 
                               metrics=['mse'])

In the cell below: 
- Train the `normalized_input_model` on normalized input (`X_train`) and output (`y_train`) 
- Set a batch size of 32 and train for 150 epochs 
- Specify the `validation_data` argument as `(X_val, y_val)` 

In [8]:
# Train the model 
normalized_input_model.fit(X_train,  
                           y_train, 
                           batch_size=32, 
                           epochs=150, 
                           validation_data=(X_val, y_val))

Epoch 1/150
33/33 [==============================] - 0s 10ms/step - loss: 179459942931078542406975488.0000 - mse: 179459942931078542406975488.0000 - val_loss: 36104148996129317453824.0000 - val_mse: 36104148996129317453824.0000
Epoch 2/150
33/33 [==============================] - 0s 6ms/step - loss: 20395251222289347248128.0000 - mse: 20395251222289347248128.0000 - val_loss: 9516518091999762448384.0000 - val_mse: 9516518091999762448384.0000
Epoch 3/150
33/33 [==============================] - 0s 5ms/step - loss: 5375886076597191573504.0000 - mse: 5375886076597191573504.0000 - val_loss: 2508411542753098334208.0000 - val_mse: 2508411542753098334208.0000
Epoch 4/150
33/33 [==============================] - 0s 5ms/step - loss: 1417003298081621409792.0000 - mse: 1417003298081621409792.0000 - val_loss: 661179653743750152192.0000 - val_mse: 661179653743750152192.0000
Epoch 5/150
33/33 [==============================] - 0s 5ms/step - loss: 373500827808902938624.0000 - mse: 37350082780890293862

33/33 [==============================] - 0s 6ms/step - loss: 6179028992.0000 - mse: 6179028992.0000 - val_loss: 5271587328.0000 - val_mse: 5271587328.0000
Epoch 47/150
33/33 [==============================] - 0s 5ms/step - loss: 6180219392.0000 - mse: 6180219392.0000 - val_loss: 5267932160.0000 - val_mse: 5267932160.0000
Epoch 48/150
33/33 [==============================] - 0s 6ms/step - loss: 6178449408.0000 - mse: 6178449408.0000 - val_loss: 5263453696.0000 - val_mse: 5263453696.0000
Epoch 49/150
33/33 [==============================] - 0s 6ms/step - loss: 6179265024.0000 - mse: 6179265024.0000 - val_loss: 5261886976.0000 - val_mse: 5261886976.0000
Epoch 50/150
33/33 [==============================] - 0s 5ms/step - loss: 6179059712.0000 - mse: 6179059712.0000 - val_loss: 5264471552.0000 - val_mse: 5264471552.0000
Epoch 51/150
33/33 [==============================] - 0s 6ms/step - loss: 6177974784.0000 - mse: 6177974784.0000 - val_loss: 5271185408.0000 - val_mse: 5271185408.0000
Epoch

Epoch 95/150
33/33 [==============================] - 0s 5ms/step - loss: 6178461696.0000 - mse: 6178461696.0000 - val_loss: 5265377792.0000 - val_mse: 5265377792.0000
Epoch 96/150
33/33 [==============================] - 0s 6ms/step - loss: 6178489856.0000 - mse: 6178489856.0000 - val_loss: 5269804032.0000 - val_mse: 5269804032.0000
Epoch 97/150
33/33 [==============================] - 0s 5ms/step - loss: 6180399104.0000 - mse: 6180399104.0000 - val_loss: 5267609088.0000 - val_mse: 5267609088.0000
Epoch 98/150
33/33 [==============================] - 0s 6ms/step - loss: 6179146240.0000 - mse: 6179146240.0000 - val_loss: 5272151040.0000 - val_mse: 5272151040.0000
Epoch 99/150
33/33 [==============================] - 0s 5ms/step - loss: 6179275776.0000 - mse: 6179275776.0000 - val_loss: 5272109568.0000 - val_mse: 5272109568.0000
Epoch 100/150
33/33 [==============================] - 0s 6ms/step - loss: 6179697152.0000 - mse: 6179697152.0000 - val_loss: 5270382080.0000 - val_mse: 5270382

33/33 [==============================] - 0s 5ms/step - loss: 6179551232.0000 - mse: 6179551232.0000 - val_loss: 5271683584.0000 - val_mse: 5271683584.0000
Epoch 144/150
33/33 [==============================] - 0s 5ms/step - loss: 6178355200.0000 - mse: 6178355200.0000 - val_loss: 5269666816.0000 - val_mse: 5269666816.0000
Epoch 145/150
33/33 [==============================] - 0s 6ms/step - loss: 6179182592.0000 - mse: 6179182592.0000 - val_loss: 5275663360.0000 - val_mse: 5275663360.0000
Epoch 146/150
33/33 [==============================] - 0s 5ms/step - loss: 6179024384.0000 - mse: 6179024384.0000 - val_loss: 5272173056.0000 - val_mse: 5272173056.0000
Epoch 147/150
33/33 [==============================] - 0s 5ms/step - loss: 6179375616.0000 - mse: 6179375616.0000 - val_loss: 5265117184.0000 - val_mse: 5265117184.0000
Epoch 148/150
33/33 [==============================] - 0s 5ms/step - loss: 6178222080.0000 - mse: 6178222080.0000 - val_loss: 5270752256.0000 - val_mse: 5270752256.0000


> _**Note that you still haven't achieved convergence! From here, it's time to normalize the output data.**_

## Normalizing the output

Again, use `StandardScaler()` to: 

- Fit and transform `y_train` 
- Transform `y_val` and `y_test` 

In [9]:
# Instantiate StandardScaler
ss_y = StandardScaler()

# Fit and transform train labels
y_train_scaled = ss_y.fit_transform(y_train)

# Transform validate and test labels
y_val_scaled = ss_y.transform(y_val)
y_test_scaled = ss_y.transform(y_test)

In the cell below: 
- Train the `normalized_model` on normalized input (`X_train`) and output (`y_train_scaled`) 
- Set a batch size of 32 and train for 150 epochs 
- Specify the `validation_data` as `(X_val, y_val_scaled)` 

In [10]:
# Model with all normalized inputs and outputs
np.random.seed(123)
normalized_model = Sequential()
normalized_model.add(layers.Dense(100, activation='relu', input_shape=(n_features,)))
normalized_model.add(layers.Dense(50, activation='relu'))
normalized_model.add(layers.Dense(1, activation='linear'))

# Compile the model
normalized_model.compile(optimizer='SGD', 
                         loss='mse', 
                         metrics=['mse']) 

# Train the model
normalized_model.fit(X_train, 
                     y_train_scaled, 
                     batch_size=32, 
                     epochs=150, 
                     validation_data=(X_val, y_val_scaled))

Epoch 1/150
33/33 [==============================] - 0s 11ms/step - loss: 0.5515 - mse: 0.5515 - val_loss: 0.2358 - val_mse: 0.2358
Epoch 2/150
33/33 [==============================] - 0s 6ms/step - loss: 0.2578 - mse: 0.2578 - val_loss: 0.1866 - val_mse: 0.1866
Epoch 3/150
33/33 [==============================] - 0s 6ms/step - loss: 0.2043 - mse: 0.2043 - val_loss: 0.1830 - val_mse: 0.1830
Epoch 4/150
33/33 [==============================] - 0s 5ms/step - loss: 0.1751 - mse: 0.1751 - val_loss: 0.1610 - val_mse: 0.1610
Epoch 5/150
33/33 [==============================] - 0s 5ms/step - loss: 0.1550 - mse: 0.1550 - val_loss: 0.1512 - val_mse: 0.1512
Epoch 6/150
33/33 [==============================] - 0s 5ms/step - loss: 0.1358 - mse: 0.1358 - val_loss: 0.1524 - val_mse: 0.1524
Epoch 7/150
33/33 [==============================] - 0s 5ms/step - loss: 0.1256 - mse: 0.1256 - val_loss: 0.1560 - val_mse: 0.1560
Epoch 8/150
33/33 [==============================] - 0s 5ms/step - loss: 0.1201 - 

33/33 [==============================] - 0s 5ms/step - loss: 0.0247 - mse: 0.0247 - val_loss: 0.2085 - val_mse: 0.2085
Epoch 64/150
33/33 [==============================] - 0s 5ms/step - loss: 0.0248 - mse: 0.0248 - val_loss: 0.2106 - val_mse: 0.2106
Epoch 65/150
33/33 [==============================] - 0s 5ms/step - loss: 0.0240 - mse: 0.0240 - val_loss: 0.2083 - val_mse: 0.2083
Epoch 66/150
33/33 [==============================] - 0s 6ms/step - loss: 0.0239 - mse: 0.0239 - val_loss: 0.2078 - val_mse: 0.2078
Epoch 67/150
33/33 [==============================] - 0s 6ms/step - loss: 0.0234 - mse: 0.0234 - val_loss: 0.2118 - val_mse: 0.2118
Epoch 68/150
33/33 [==============================] - 0s 5ms/step - loss: 0.0233 - mse: 0.0233 - val_loss: 0.2088 - val_mse: 0.2088
Epoch 69/150
33/33 [==============================] - 0s 5ms/step - loss: 0.0227 - mse: 0.0227 - val_loss: 0.2086 - val_mse: 0.2086
Epoch 70/150
33/33 [==============================] - 0s 5ms/step - loss: 0.0228 - mse: 0

Epoch 125/150
33/33 [==============================] - 0s 5ms/step - loss: 0.0123 - mse: 0.0123 - val_loss: 0.2242 - val_mse: 0.2242
Epoch 126/150
33/33 [==============================] - 0s 5ms/step - loss: 0.0120 - mse: 0.0120 - val_loss: 0.2257 - val_mse: 0.2257
Epoch 127/150
33/33 [==============================] - 0s 5ms/step - loss: 0.0119 - mse: 0.0119 - val_loss: 0.2253 - val_mse: 0.2253
Epoch 128/150
33/33 [==============================] - 0s 5ms/step - loss: 0.0119 - mse: 0.0119 - val_loss: 0.2251 - val_mse: 0.2251
Epoch 129/150
33/33 [==============================] - 0s 5ms/step - loss: 0.0116 - mse: 0.0116 - val_loss: 0.2249 - val_mse: 0.2249
Epoch 130/150
33/33 [==============================] - 0s 5ms/step - loss: 0.0116 - mse: 0.0116 - val_loss: 0.2265 - val_mse: 0.2265
Epoch 131/150
33/33 [==============================] - 0s 5ms/step - loss: 0.0116 - mse: 0.0116 - val_loss: 0.2270 - val_mse: 0.2270
Epoch 132/150
33/33 [==============================] - 0s 5ms/step - 

Nicely done! After normalizing both the input and output, the model finally converged. 

- Evaluate the model (`normalized_model`) on training data (`X_train` and `y_train_scaled`) 

In [11]:
# Evaluate the model on training data
normalized_model.evaluate(X_train, y_train_scaled)

33/33 [==============================] - 0s 3ms/step - loss: 0.0095 - mse: 0.0095


[0.009506430476903915, 0.009506430476903915]

- Evaluate the model (`normalized_model`) on validate data (`X_val` and `y_val_scaled`) 

In [12]:
# Evaluate the model on validate data
normalized_model.evaluate(X_val, y_val_scaled)

9/9 [==============================] - 0s 3ms/step - loss: 0.2289 - mse: 0.2289


[0.22886480391025543, 0.22886480391025543]

Since the output is normalized, the metric above is not interpretable. To remedy this: 

- Generate predictions on validate data (`X_val`) 
- Transform these predictions back to original scale using `ss_y` 
- Now you can calculate the RMSE in the original units with `y_val` and `y_val_pred` 

In [13]:
# Generate predictions on validate data
y_val_pred_scaled = normalized_model.predict(X_val)

# Transform the predictions back to original scale
y_val_pred = ss_y.inverse_transform(y_val_pred_scaled)

# RMSE of validate data
np.sqrt(mean_squared_error(y_val, y_val_pred))

37594.21305743835

Great! Now that you have a converged model, you can also experiment with alternative optimizers and initialization strategies to see if you can find a better global minimum. (After all, the current models may have converged to a local minimum.) 

## Using Weight Initializers

In this section you will to use alternative initialization and optimization strategies. At the end, you'll then be asked to select the model which you believe performs the best.  

##  He Initialization

In the cell below, sepcify the following in the first hidden layer:  
  - 100 units 
  - `'relu'` activation 
  - `input_shape` 
  - `kernel_initializer='he_normal'`  

In [14]:
np.random.seed(123)
he_model = Sequential()

# Add the first hidden layer
he_model.add(layers.Dense(100, kernel_initializer='he_normal', activation='relu', input_shape=(n_features,)))

# Add another hidden layer
he_model.add(layers.Dense(50, activation='relu'))

# Add an output layer
he_model.add(layers.Dense(1, activation='linear'))

# Compile the model
he_model.compile(optimizer='SGD', 
                 loss='mse', 
                 metrics=['mse'])

# Train the model
he_model.fit(X_train, 
             y_train_scaled, 
             batch_size=32, 
             epochs=150, 
             validation_data=(X_val, y_val_scaled))

Epoch 1/150
33/33 [==============================] - 0s 10ms/step - loss: 0.4488 - mse: 0.4488 - val_loss: 0.2605 - val_mse: 0.2605
Epoch 2/150
33/33 [==============================] - 0s 5ms/step - loss: 0.2453 - mse: 0.2453 - val_loss: 0.1864 - val_mse: 0.1864
Epoch 3/150
33/33 [==============================] - 0s 5ms/step - loss: 0.2016 - mse: 0.2016 - val_loss: 0.1694 - val_mse: 0.1694
Epoch 4/150
33/33 [==============================] - 0s 5ms/step - loss: 0.1739 - mse: 0.1739 - val_loss: 0.1653 - val_mse: 0.1653
Epoch 5/150
33/33 [==============================] - 0s 5ms/step - loss: 0.1551 - mse: 0.1551 - val_loss: 0.1573 - val_mse: 0.1573
Epoch 6/150
33/33 [==============================] - 0s 5ms/step - loss: 0.1422 - mse: 0.1422 - val_loss: 0.1606 - val_mse: 0.1606
Epoch 7/150
33/33 [==============================] - 0s 5ms/step - loss: 0.1292 - mse: 0.1292 - val_loss: 0.1535 - val_mse: 0.1535
Epoch 8/150
33/33 [==============================] - 0s 5ms/step - loss: 0.1210 - 

33/33 [==============================] - 0s 5ms/step - loss: 0.0233 - mse: 0.0233 - val_loss: 0.1674 - val_mse: 0.1674
Epoch 64/150
33/33 [==============================] - 0s 5ms/step - loss: 0.0235 - mse: 0.0235 - val_loss: 0.1707 - val_mse: 0.1707
Epoch 65/150
33/33 [==============================] - 0s 5ms/step - loss: 0.0230 - mse: 0.0230 - val_loss: 0.1667 - val_mse: 0.1667
Epoch 66/150
33/33 [==============================] - 0s 5ms/step - loss: 0.0224 - mse: 0.0224 - val_loss: 0.1660 - val_mse: 0.1660
Epoch 67/150
33/33 [==============================] - 0s 5ms/step - loss: 0.0227 - mse: 0.0227 - val_loss: 0.1696 - val_mse: 0.1696
Epoch 68/150
33/33 [==============================] - 0s 5ms/step - loss: 0.0221 - mse: 0.0221 - val_loss: 0.1674 - val_mse: 0.1674
Epoch 69/150
33/33 [==============================] - 0s 5ms/step - loss: 0.0215 - mse: 0.0215 - val_loss: 0.1695 - val_mse: 0.1695
Epoch 70/150
33/33 [==============================] - 0s 5ms/step - loss: 0.0214 - mse: 0

Epoch 125/150
33/33 [==============================] - 0s 5ms/step - loss: 0.0109 - mse: 0.0109 - val_loss: 0.1742 - val_mse: 0.1742
Epoch 126/150
33/33 [==============================] - 0s 5ms/step - loss: 0.0108 - mse: 0.0108 - val_loss: 0.1765 - val_mse: 0.1765
Epoch 127/150
33/33 [==============================] - 0s 5ms/step - loss: 0.0105 - mse: 0.0105 - val_loss: 0.1780 - val_mse: 0.1780
Epoch 128/150
33/33 [==============================] - 0s 5ms/step - loss: 0.0106 - mse: 0.0106 - val_loss: 0.1756 - val_mse: 0.1756
Epoch 129/150
33/33 [==============================] - 0s 5ms/step - loss: 0.0105 - mse: 0.0105 - val_loss: 0.1762 - val_mse: 0.1762
Epoch 130/150
33/33 [==============================] - 0s 5ms/step - loss: 0.0103 - mse: 0.0103 - val_loss: 0.1769 - val_mse: 0.1769
Epoch 131/150
33/33 [==============================] - 0s 5ms/step - loss: 0.0102 - mse: 0.0102 - val_loss: 0.1783 - val_mse: 0.1783
Epoch 132/150
33/33 [==============================] - 0s 5ms/step - 

Evaluate the model (`he_model`) on training data (`X_train` and `y_train_scaled`) 

In [15]:
# Evaluate the model on training data
he_model.evaluate(X_train, y_train_scaled)

33/33 [==============================] - 0s 4ms/step - loss: 0.0080 - mse: 0.0080


[0.008021695539355278, 0.008021695539355278]

Evaluate the model (`he_model`) on validate data (`X_train` and `y_train_scaled`) 

In [16]:
# Evaluate the model on validate data
he_model.evaluate(X_val, y_val_scaled)

9/9 [==============================] - 0s 3ms/step - loss: 0.1814 - mse: 0.1814


[0.18140417337417603, 0.18140417337417603]

## Lecun Initialization 

In the cell below, sepcify the following in the first hidden layer:  
  - 100 units 
  - `'relu'` activation 
  - `input_shape` 
  - `kernel_initializer='lecun_normal'` 

In [17]:
np.random.seed(123)
lecun_model = Sequential()

# Add the first hidden layer
lecun_model.add(layers.Dense(100, kernel_initializer='lecun_normal', activation='relu', input_shape=(n_features,)))

# Add another hidden layer
lecun_model.add(layers.Dense(50, activation='relu'))

# Add an output layer
lecun_model.add(layers.Dense(1, activation='linear'))

# Compile the model
lecun_model.compile(optimizer='SGD', 
                    loss='mse', 
                    metrics=['mse'])

# Train the model
lecun_model.fit(X_train, 
                y_train_scaled, 
                batch_size=32, 
                epochs=150, 
                validation_data=(X_val, y_val_scaled))

Epoch 1/150
33/33 [==============================] - 0s 10ms/step - loss: 0.3904 - mse: 0.3904 - val_loss: 0.1599 - val_mse: 0.1599
Epoch 2/150
33/33 [==============================] - 0s 6ms/step - loss: 0.2080 - mse: 0.2080 - val_loss: 0.1320 - val_mse: 0.1320
Epoch 3/150
33/33 [==============================] - 0s 5ms/step - loss: 0.1721 - mse: 0.1721 - val_loss: 0.1195 - val_mse: 0.1195
Epoch 4/150
33/33 [==============================] - 0s 5ms/step - loss: 0.1506 - mse: 0.1506 - val_loss: 0.1102 - val_mse: 0.1102
Epoch 5/150
33/33 [==============================] - 0s 5ms/step - loss: 0.1338 - mse: 0.1338 - val_loss: 0.1088 - val_mse: 0.1088
Epoch 6/150
33/33 [==============================] - 0s 5ms/step - loss: 0.1223 - mse: 0.1223 - val_loss: 0.1047 - val_mse: 0.1047
Epoch 7/150
33/33 [==============================] - 0s 5ms/step - loss: 0.1109 - mse: 0.1109 - val_loss: 0.1057 - val_mse: 0.1057
Epoch 8/150
33/33 [==============================] - 0s 5ms/step - loss: 0.1019 - 

33/33 [==============================] - 0s 5ms/step - loss: 0.0235 - mse: 0.0235 - val_loss: 0.1045 - val_mse: 0.1045
Epoch 62/150
33/33 [==============================] - 0s 5ms/step - loss: 0.0226 - mse: 0.0226 - val_loss: 0.1044 - val_mse: 0.1044
Epoch 63/150
33/33 [==============================] - 0s 5ms/step - loss: 0.0230 - mse: 0.0230 - val_loss: 0.1039 - val_mse: 0.1039
Epoch 64/150
33/33 [==============================] - 0s 5ms/step - loss: 0.0221 - mse: 0.0221 - val_loss: 0.1038 - val_mse: 0.1038
Epoch 65/150
33/33 [==============================] - 0s 5ms/step - loss: 0.0220 - mse: 0.0220 - val_loss: 0.1048 - val_mse: 0.1048
Epoch 66/150
33/33 [==============================] - 0s 5ms/step - loss: 0.0217 - mse: 0.0217 - val_loss: 0.1079 - val_mse: 0.1079
Epoch 67/150
33/33 [==============================] - 0s 5ms/step - loss: 0.0218 - mse: 0.0218 - val_loss: 0.1054 - val_mse: 0.1054
Epoch 68/150
33/33 [==============================] - 0s 5ms/step - loss: 0.0214 - mse: 0

Epoch 123/150
33/33 [==============================] - 0s 5ms/step - loss: 0.0116 - mse: 0.0116 - val_loss: 0.1072 - val_mse: 0.1072
Epoch 124/150
33/33 [==============================] - 0s 5ms/step - loss: 0.0117 - mse: 0.0117 - val_loss: 0.1071 - val_mse: 0.1071
Epoch 125/150
33/33 [==============================] - 0s 5ms/step - loss: 0.0117 - mse: 0.0117 - val_loss: 0.1074 - val_mse: 0.1074
Epoch 126/150
33/33 [==============================] - 0s 5ms/step - loss: 0.0113 - mse: 0.0113 - val_loss: 0.1066 - val_mse: 0.1066
Epoch 127/150
33/33 [==============================] - 0s 5ms/step - loss: 0.0112 - mse: 0.0112 - val_loss: 0.1054 - val_mse: 0.1054
Epoch 128/150
33/33 [==============================] - 0s 5ms/step - loss: 0.0109 - mse: 0.0109 - val_loss: 0.1077 - val_mse: 0.1077
Epoch 129/150
33/33 [==============================] - 0s 5ms/step - loss: 0.0111 - mse: 0.0111 - val_loss: 0.1069 - val_mse: 0.1069
Epoch 130/150
33/33 [==============================] - 0s 5ms/step - 

Evaluate the model (`lecun_model`) on training data (`X_train` and `y_train_scaled`) 

In [18]:
# Evaluate the model on training data
lecun_model.evaluate(X_train, y_train_scaled)

33/33 [==============================] - 0s 4ms/step - loss: 0.0098 - mse: 0.0098


[0.009834765456616879, 0.009834765456616879]

Evaluate the model (`lecun_model`) on validate data (`X_train` and `y_train_scaled`) 

In [19]:
# Evaluate the model on validate data
lecun_model.evaluate(X_val, y_val_scaled)

9/9 [==============================] - 0s 4ms/step - loss: 0.1091 - mse: 0.1091


[0.10906457901000977, 0.10906457901000977]

Not much of a difference, but a useful note to consider when tuning your network. Next, let's investigate the impact of various optimization algorithms.

## RMSprop 

Compile the `rmsprop_model` with: 

- `'rmsprop'` as the optimizer 
- track `'mse'` as the loss and metric  

In [20]:
np.random.seed(123)
rmsprop_model = Sequential()
rmsprop_model.add(layers.Dense(100, activation='relu', input_shape=(n_features,)))
rmsprop_model.add(layers.Dense(50, activation='relu'))
rmsprop_model.add(layers.Dense(1, activation='linear'))

# Compile the model
rmsprop_model.compile(optimizer='rmsprop', 
                      loss='mse', 
                      metrics=['mse'])

# Train the model
rmsprop_model.fit(X_train, 
                  y_train_scaled, 
                  batch_size=32, 
                  epochs=150, 
                  validation_data=(X_val, y_val_scaled))

Epoch 1/150
33/33 [==============================] - 0s 10ms/step - loss: 0.3615 - mse: 0.3615 - val_loss: 0.1160 - val_mse: 0.1160
Epoch 2/150
33/33 [==============================] - 0s 6ms/step - loss: 0.1791 - mse: 0.1791 - val_loss: 0.1079 - val_mse: 0.1079
Epoch 3/150
33/33 [==============================] - 0s 6ms/step - loss: 0.1279 - mse: 0.1279 - val_loss: 0.0947 - val_mse: 0.0947
Epoch 4/150
33/33 [==============================] - 0s 6ms/step - loss: 0.1099 - mse: 0.1099 - val_loss: 0.0980 - val_mse: 0.0980
Epoch 5/150
33/33 [==============================] - 0s 6ms/step - loss: 0.0842 - mse: 0.0842 - val_loss: 0.0953 - val_mse: 0.0953
Epoch 6/150
33/33 [==============================] - 0s 6ms/step - loss: 0.0652 - mse: 0.0652 - val_loss: 0.1418 - val_mse: 0.1418
Epoch 7/150
33/33 [==============================] - 0s 6ms/step - loss: 0.0631 - mse: 0.0631 - val_loss: 0.1098 - val_mse: 0.1098
Epoch 8/150
33/33 [==============================] - 0s 6ms/step - loss: 0.0552 - 

33/33 [==============================] - 0s 6ms/step - loss: 0.0065 - mse: 0.0065 - val_loss: 0.0866 - val_mse: 0.0866
Epoch 64/150
33/33 [==============================] - 0s 5ms/step - loss: 0.0092 - mse: 0.0092 - val_loss: 0.0981 - val_mse: 0.0981
Epoch 65/150
33/33 [==============================] - 0s 6ms/step - loss: 0.0105 - mse: 0.0105 - val_loss: 0.0953 - val_mse: 0.0953
Epoch 66/150
33/33 [==============================] - 0s 6ms/step - loss: 0.0092 - mse: 0.0092 - val_loss: 0.0959 - val_mse: 0.0959
Epoch 67/150
33/33 [==============================] - 0s 6ms/step - loss: 0.0074 - mse: 0.0074 - val_loss: 0.0936 - val_mse: 0.0936
Epoch 68/150
33/33 [==============================] - 0s 5ms/step - loss: 0.0078 - mse: 0.0078 - val_loss: 0.0958 - val_mse: 0.0958
Epoch 69/150
33/33 [==============================] - 0s 5ms/step - loss: 0.0068 - mse: 0.0068 - val_loss: 0.0898 - val_mse: 0.0898
Epoch 70/150
33/33 [==============================] - 0s 5ms/step - loss: 0.0101 - mse: 0

Epoch 125/150
33/33 [==============================] - 0s 5ms/step - loss: 0.0055 - mse: 0.0055 - val_loss: 0.0890 - val_mse: 0.0890
Epoch 126/150
33/33 [==============================] - 0s 5ms/step - loss: 0.0042 - mse: 0.0042 - val_loss: 0.0810 - val_mse: 0.0810
Epoch 127/150
33/33 [==============================] - 0s 5ms/step - loss: 0.0056 - mse: 0.0056 - val_loss: 0.1011 - val_mse: 0.1011
Epoch 128/150
33/33 [==============================] - 0s 5ms/step - loss: 0.0048 - mse: 0.0048 - val_loss: 0.0778 - val_mse: 0.0778
Epoch 129/150
33/33 [==============================] - 0s 6ms/step - loss: 0.0045 - mse: 0.0045 - val_loss: 0.0980 - val_mse: 0.0980
Epoch 130/150
33/33 [==============================] - 0s 6ms/step - loss: 0.0076 - mse: 0.0076 - val_loss: 0.0907 - val_mse: 0.0907
Epoch 131/150
33/33 [==============================] - 0s 5ms/step - loss: 0.0033 - mse: 0.0033 - val_loss: 0.0973 - val_mse: 0.0973
Epoch 132/150
33/33 [==============================] - 0s 5ms/step - 

Evaluate the model (`rmsprop_model`) on training data (`X_train` and `y_train_scaled`) 

In [21]:
# Evaluate the model on training data
rmsprop_model.evaluate(X_train, y_train_scaled)

33/33 [==============================] - 0s 4ms/step - loss: 0.0042 - mse: 0.0042


[0.004239383619278669, 0.004239383619278669]

Evaluate the model (`rmsprop_model`) on training data (`X_train` and `y_train_scaled`) 

In [22]:
# Evaluate the model on validate data
rmsprop_model.evaluate(X_val, y_val_scaled)

9/9 [==============================] - 0s 4ms/step - loss: 0.0889 - mse: 0.0889


[0.08894751965999603, 0.08894751965999603]

## Adam 

Compile the `adam_model` with: 

- `'Adam'` as the optimizer 
- track `'mse'` as the loss and metric  

In [23]:
np.random.seed(123)
adam_model = Sequential()
adam_model.add(layers.Dense(100, activation='relu', input_shape=(n_features,)))
adam_model.add(layers.Dense(50, activation='relu'))
adam_model.add(layers.Dense(1, activation='linear'))

# Compile the model
adam_model.compile(optimizer='Adam', 
                   loss='mse', 
                   metrics=['mse'])

# Train the model
adam_model.fit(X_train, 
               y_train_scaled, 
               batch_size=32, 
               epochs=150, 
               validation_data=(X_val, y_val_scaled))

Epoch 1/150
33/33 [==============================] - 0s 14ms/step - loss: 0.5695 - mse: 0.5695 - val_loss: 0.1813 - val_mse: 0.1813
Epoch 2/150
33/33 [==============================] - 0s 6ms/step - loss: 0.2052 - mse: 0.2052 - val_loss: 0.1370 - val_mse: 0.1370
Epoch 3/150
33/33 [==============================] - 0s 5ms/step - loss: 0.1362 - mse: 0.1362 - val_loss: 0.1306 - val_mse: 0.1306
Epoch 4/150
33/33 [==============================] - 0s 5ms/step - loss: 0.1087 - mse: 0.1087 - val_loss: 0.1307 - val_mse: 0.1307
Epoch 5/150
33/33 [==============================] - 0s 6ms/step - loss: 0.0793 - mse: 0.0793 - val_loss: 0.1322 - val_mse: 0.1322
Epoch 6/150
33/33 [==============================] - 0s 5ms/step - loss: 0.0607 - mse: 0.0607 - val_loss: 0.1374 - val_mse: 0.1374
Epoch 7/150
33/33 [==============================] - 0s 5ms/step - loss: 0.0527 - mse: 0.0527 - val_loss: 0.1623 - val_mse: 0.1623
Epoch 8/150
33/33 [==============================] - 0s 7ms/step - loss: 0.0399 - 

33/33 [==============================] - 0s 5ms/step - loss: 0.0045 - mse: 0.0045 - val_loss: 0.1653 - val_mse: 0.1653
Epoch 64/150
33/33 [==============================] - 0s 5ms/step - loss: 0.0044 - mse: 0.0044 - val_loss: 0.1476 - val_mse: 0.1476
Epoch 65/150
33/33 [==============================] - 0s 5ms/step - loss: 0.0057 - mse: 0.0057 - val_loss: 0.1542 - val_mse: 0.1542
Epoch 66/150
33/33 [==============================] - 0s 5ms/step - loss: 0.0049 - mse: 0.0049 - val_loss: 0.1493 - val_mse: 0.1493
Epoch 67/150
33/33 [==============================] - 0s 5ms/step - loss: 0.0047 - mse: 0.0047 - val_loss: 0.1464 - val_mse: 0.1464
Epoch 68/150
33/33 [==============================] - 0s 6ms/step - loss: 0.0041 - mse: 0.0041 - val_loss: 0.1495 - val_mse: 0.1495
Epoch 69/150
33/33 [==============================] - 0s 5ms/step - loss: 0.0048 - mse: 0.0048 - val_loss: 0.1503 - val_mse: 0.1503
Epoch 70/150
33/33 [==============================] - 0s 5ms/step - loss: 0.0036 - mse: 0

33/33 [==============================] - 0s 5ms/step - loss: 9.7620e-04 - mse: 9.7620e-04 - val_loss: 0.1280 - val_mse: 0.1280
Epoch 124/150
33/33 [==============================] - 0s 6ms/step - loss: 7.5280e-04 - mse: 7.5280e-04 - val_loss: 0.1262 - val_mse: 0.1262
Epoch 125/150
33/33 [==============================] - 0s 5ms/step - loss: 8.0427e-04 - mse: 8.0427e-04 - val_loss: 0.1292 - val_mse: 0.1292
Epoch 126/150
33/33 [==============================] - 0s 6ms/step - loss: 7.6511e-04 - mse: 7.6511e-04 - val_loss: 0.1279 - val_mse: 0.1279
Epoch 127/150
33/33 [==============================] - 0s 5ms/step - loss: 7.3097e-04 - mse: 7.3097e-04 - val_loss: 0.1289 - val_mse: 0.1289
Epoch 128/150
33/33 [==============================] - 0s 5ms/step - loss: 6.9430e-04 - mse: 6.9430e-04 - val_loss: 0.1263 - val_mse: 0.1263
Epoch 129/150
33/33 [==============================] - 0s 5ms/step - loss: 7.8019e-04 - mse: 7.8019e-04 - val_loss: 0.1279 - val_mse: 0.1279
Epoch 130/150
33/33 [======

Evaluate the model (`adam_model`) on training data (`X_train` and `y_train_scaled`) 

In [24]:
# Evaluate the model on training data
adam_model.evaluate(X_train, y_train_scaled)

33/33 [==============================] - 0s 4ms/step - loss: 0.0022 - mse: 0.0022


[0.002185358665883541, 0.0021853584330528975]

Evaluate the model (`adam_model`) on training data (`X_train` and `y_train_scaled`) 

In [25]:
# Evaluate the model on validate data
adam_model.evaluate(X_val, y_val_scaled)

9/9 [==============================] - 0s 4ms/step - loss: 0.1300 - mse: 0.1300


[0.13000035285949707, 0.13000035285949707]

## Select a Final Model

Now, select the model with the best performance based on the training and validation sets. Evaluate this top model using the test set!

In [26]:
# Evaluate the best model on test data
rmsprop_model.evaluate(X_test, y_test_scaled)

5/5 [==============================] - 0s 4ms/step - loss: 0.1347 - mse: 0.1347


[0.13466529548168182, 0.13466529548168182]

As earlier, this metric is hard to interpret because the output is scaled. 

- Generate predictions on test data (`X_test`) 
- Transform these predictions back to original scale using `ss_y` 
- Now you can calculate the RMSE in the original units with `y_test` and `y_test_pred` 

In [27]:
# Generate predictions on test data
y_test_pred_scaled = rmsprop_model.predict(X_test)

# Transform the predictions back to original scale
y_test_pred = ss_y.inverse_transform(y_test_pred_scaled)

# MSE of test data
np.sqrt(mean_squared_error(y_test, y_test_pred))

28837.61495488833

## Summary  

In this lab, you worked to ensure your model converged properly by normalizing both the input and output. Additionally, you also investigated the impact of varying initialization and optimization routines.